# Financial Calculator
## A few tips
* To execute a cell, click in it and then type [shift] + [enter] or pressing the ▶ button in the toolbar above
* If you are stuck, use ◼ in the toolbar to stop! Or press [I],[I]
* Press [TAB] to auto-complete Python code
* If you need help, just reach out to any of us!

## Launching a python program!

What we have experienced in the Introduction module allow us to run python code on the fly.  In reality, python programs are ran standalone. Let's try running a python program in that way!

Running the script below will launch the actual program!

In [ ]:
%%bash
source venv/bin/activate

python main.py > /dev/null 2>&1

##### You should be able to see the screen as below

![Start of the financial calculator program](images/financial_cal_start.png)

## Try it out!

### Let's select the Freedom Fund
#### Imagine you put HKD1000 of your savings as initial deposit, and contribute HKD150 monthly into the fund.  With the investment time of 10 years, how much will you get?

That's right, HKD 188,345.93

## Question!
### If you want to have HKD1,000,000 or above, what's the minimum number of years you will need to invest?

#### 5 additional years, where the future balance will become HKD1,080,778.8!
#### You make the animated heart appears!
![Heart](web/assets/heartbeat.gif)

## Question!

### What parameters can you change to make the animated heart disappears

1) Select another fund with higher or lower rate of return? 
<br> 2) Place more or less initial deposits?
<br> 3) Place more or less contributions?
<br> 4) Do monthly or annually contributions instead?
<br> 5) Decrease or increase your investment time span?
<br> 6) Have daily, monthly or annually compund frequency instead?

While it make take you a while to figure this out, a computer program can solve your problem quickly, try it out!

## Challenge!

### Are you able to add a new fund and change the interest rate?

#### TIPS: Notice Line 35-39 below. 

Are you able to make some changes there in order to add your own named fund?


After you made the edit, running the cell would save your changes. Then you can rerun the program again to see if it pick up your changes!

In [3]:
%%writefile ./core/utils.py
from __future__ import annotations
from enum import Enum
from typing import Optional


class ActionType(Enum):
    ADD = 'add'
    SUBTRACT = 'sub'
    SET = 'set'

    @staticmethod
    def value_from(name: str) -> Optional[ActionType]:
        return next(filter(
            lambda i: i.value == name,
            (i for i in ActionType)
        ))


class Sprite(Enum):
    BIRD = 'assets/bird.gif'
    CAKE = 'assets/cake.gif'
    ELEPHANT = 'assets/elephant.gif'
    FISH = 'assets/fish.gif'
    GAME = 'assets/game.gif'
    HEARTBEAT = 'assets/heartbeat.gif'
    PONY = 'assets/pony.gif'
    SHEEP = 'assets/sheep.gif'
    SNOW_FIGHT = 'assets/snow-fight.gif'
    STARWARS = 'assets/starwars.gif'
    NONE = ''


class Fund(Enum):
    HONG_KONG_ETF = ('Hong Kong ETF', 13.5)
    FREEDOM_FUND = ('Freedom Fund', 35)
    UNION_SP500_INDEX = ('Union SP500 Index Fund', 25)
    YU_LENG_1000_ETF = ('Yu Leng 100 ETF', 15)
    TANG_5000_ETF = ('Tang 5000 Trust ETF', 18)
    Niki_8864_T=('Niki 8864 T', 27)

    @property
    def returns(self):
        return self.value[1]

    @property
    def name(self):
        return self.value[0]

    @staticmethod
    def value_from(name: str) -> Optional[Fund]:
        return next(filter(
            lambda i: i.name == name,
            (i for i in Fund)
        ))


Overwriting ./core/utils.py


## Now that you have changed the code, lets rerun!
### Close the previous application and run the code below (which is same as how we run it at the first time)

In [4]:
%%bash
source venv/bin/activate

python main.py > /dev/null 2>&1

## Another challenge!

### Are you able to change the heartbeat animation to something else?

####TIPS: Notice Line 10-18 below. 

Are you able to make some changes there in order to change the heartbeat animation to something other animation?  You might want to scroll up and refer to the last file to see what were the other animation available.

After you made the edit, running the cell would save your changes. Then you can rerun the program again to see if it pick up your changes!


In [15]:
%%writefile ./core/calculator.py
from dataclasses import dataclass
from datetime import datetime

from core.utils import Fund, Sprite, ActionType

import time
import random
import RPi.GPIO as GPIO

@dataclass
class Calculator:
    initial_deposit: float = 0
    contribution_amount: float = 0
    investment_timespan: int = 0
    fund: Fund = Fund.HONG_KONG_ETF
    future_balance: float = 0
    contribution_period: int = 12
    compound_period: int = 12
    sprite: Sprite = Sprite.HEARTBEAT
    goal: float = 1000000

        
    def set_lights(self):
        """
        Set the lights up!
        """
        LED = 17
        GPIO.setmode(GPIO.BCM)
        GPIO.setup(LED, GPIO.OUT) 

        GPIO.output(LED, True)
        time.sleep(10.0)

        GPIO.cleanup() 
        return None
    
    def get_sprite(self):
        """
        Used to retrieve the sprite displayed when target investment reached
        """
        return self.sprite.value

    def get_funds(self):
        """
        Used to retrieve a list of available funds
        :return:
        """
        return [i.value for i in Fund]

    def get_selected_fund(self):
        """
        Retrieves the current fund selected by the user
        """
        return {"name": self.fund.name, "returns": self.fund.returns}

    def get_amount(self, field: str):
        """
        Used to retrieve values from the calculator. E.g. initial deposit
        :param field: name of the field
        :return:
        """
        return getattr(self, field)

    def change_fund(self, name: str):
        """
        Change the fund selected for the calculation
        :param name: Name of the fund
        :return:
        """
        self.fund = Fund.value_from(name)

    def change_amount(self, field: str, value, action_string: str):
        """
        Used to change a property on the calculator. E.g. initial deposit, contribution amount e.t.c
        :param field: calculator property
        :param value: Any
        :param action_string: ADD | SUBTRACT | SET
        :return: the updated field value
        """
        action = ActionType.value_from(action_string)
        change = getattr(self, field)

        if action == ActionType.ADD:
            change += value
        elif action == ActionType.SUBTRACT:
            change -= value
        elif action == ActionType.SET:
            change = value

        if change >= 0:
            setattr(self, field, change)
            return change
        return getattr(self, field)

    def calculate_future_values(self):
        """
        Calculates the future value
        """
        current_year = datetime.now().year
        labels = [year for year in range(current_year, current_year + int(self.investment_timespan))]
        principal_dataset = {
            "label": 'Total Principal',
            "backgroundColor": 'rgb(0, 123, 255)',
            "data": []
        }
        earnings_dataset = {
            "label": "Total Earnings",
            "backgroundColor": 'rgb(23, 162, 184)',
            "data": []
        }

        self.future_balance = 0

        for time in range(1, self.investment_timespan + 1):
            principal = self.initial_deposit + (self.contribution_amount * self.contribution_period * time)
            earnings = 0
            balance = principal
            estimated_return = self.fund.returns / 100

            if estimated_return:
                gains = (1 + estimated_return / self.compound_period) ** (self.compound_period * time)
                compound_earnings = self.initial_deposit * gains
                contribution_earnings = self.contribution_amount * (gains - 1) / (estimated_return / self.contribution_period)
                earnings = compound_earnings + contribution_earnings - principal
                balance = compound_earnings + contribution_earnings

            self.future_balance = balance
            principal_dataset['data'].append(round(principal, 2))
            earnings_dataset['data'].append(round(earnings, 2))

        return {
            "labels": labels,
            "datasets": [principal_dataset, earnings_dataset],
        }


Overwriting ./core/calculator.py


In [16]:
%%bash
source venv/bin/activate

python main.py > /dev/null 2>&1

## Big Challenge! 
If you have completed the electronics module, come back here for the big challenge!

### Apart from the heartbeat animation, can you cause the LED lights to blink?

#### TIPS: Reuse the code you learnt in the electronics module!  Refer to line 24-36 and see if its something familiar!

After you made the edit, running the cell would save your changes. Then you can rerun the program again to see if it pick up your changes!

In [ ]:
%%writefile ./core/calculator.py
from dataclasses import dataclass
from datetime import datetime

from core.utils import Fund, Sprite, ActionType

import time
import random
import RPi.GPIO as GPIO

@dataclass
class Calculator:
    initial_deposit: float = 0
    contribution_amount: float = 0
    investment_timespan: int = 0
    fund: Fund = Fund.HONG_KONG_ETF
    future_balance: float = 0
    contribution_period: int = 12
    compound_period: int = 12
    sprite: Sprite = Sprite.HEARTBEAT
    goal: float = 1000000

        
    def set_lights(self):
        """
        Set the lights up!
        """
        #LED = 17
        #GPIO.setmode(GPIO.BCM)
        #GPIO.setup(LED, GPIO.OUT) 

        #GPIO.output(LED, True)
        #time.sleep(10.0)

        #GPIO.cleanup() 
        return None
    
    def get_sprite(self):
        """
        Used to retrieve the sprite displayed when target investment reached
        """
        return self.sprite.value

    def get_funds(self):
        """
        Used to retrieve a list of available funds
        :return:
        """
        return [i.value for i in Fund]

    def get_selected_fund(self):
        """
        Retrieves the current fund selected by the user
        """
        return {"name": self.fund.name, "returns": self.fund.returns}

    def get_amount(self, field: str):
        """
        Used to retrieve values from the calculator. E.g. initial deposit
        :param field: name of the field
        :return:
        """
        return getattr(self, field)

    def change_fund(self, name: str):
        """
        Change the fund selected for the calculation
        :param name: Name of the fund
        :return:
        """
        self.fund = Fund.value_from(name)

    def change_amount(self, field: str, value, action_string: str):
        """
        Used to change a property on the calculator. E.g. initial deposit, contribution amount e.t.c
        :param field: calculator property
        :param value: Any
        :param action_string: ADD | SUBTRACT | SET
        :return: the updated field value
        """
        action = ActionType.value_from(action_string)
        change = getattr(self, field)

        if action == ActionType.ADD:
            change += value
        elif action == ActionType.SUBTRACT:
            change -= value
        elif action == ActionType.SET:
            change = value

        if change >= 0:
            setattr(self, field, change)
            return change
        return getattr(self, field)

    def calculate_future_values(self):
        """
        Calculates the future value
        """
        current_year = datetime.now().year
        labels = [year for year in range(current_year, current_year + int(self.investment_timespan))]
        principal_dataset = {
            "label": 'Total Principal',
            "backgroundColor": 'rgb(0, 123, 255)',
            "data": []
        }
        earnings_dataset = {
            "label": "Total Earnings",
            "backgroundColor": 'rgb(23, 162, 184)',
            "data": []
        }

        self.future_balance = 0

        for time in range(1, self.investment_timespan + 1):
            principal = self.initial_deposit + (self.contribution_amount * self.contribution_period * time)
            earnings = 0
            balance = principal
            estimated_return = self.fund.returns / 100

            if estimated_return:
                gains = (1 + estimated_return / self.compound_period) ** (self.compound_period * time)
                compound_earnings = self.initial_deposit * gains
                contribution_earnings = self.contribution_amount * (gains - 1) / (estimated_return / self.contribution_period)
                earnings = compound_earnings + contribution_earnings - principal
                balance = compound_earnings + contribution_earnings

            self.future_balance = balance
            principal_dataset['data'].append(round(principal, 2))
            earnings_dataset['data'].append(round(earnings, 2))

        return {
            "labels": labels,
            "datasets": [principal_dataset, earnings_dataset],
        }


In [44]:
%%bash
source venv/bin/activate

python main.py > /dev/null 2>&1

# Congratulations! Done! On to more programming!